Reference: https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf, https://zhuanlan.zhihu.com/p/426978026

reduce2: Avoid bank conflict in reduce1.  
reduce7: The final step has been replaced by utilizing a shuffle operation, but no significant performance enhancement has been observed.  
reduce8: Due to the global memory and bank conflict restriction, we cannot process data with divid-and-conquer strategy. We can just set block size equal to warp size for avoiding bank conflict.

In [ ]:
%%writefile reduction.cu
#include <stdio.h>
#include <stdlib.h>
#include <type_traits>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/sequence.h>

#define TYPE int
#define N 42990
#define BLOCK_SIZE 1024
#define NUM_PER_THREAD 8
#define WARP_SIZE (BLOCK_SIZE / NUM_PER_THREAD / 32)

__global__ void  warm_up()
{
    int indexX = threadIdx.x + blockIdx.x * blockDim.x;
    if (indexX < N)
    {
        float a = 0.0f;
        float b = 1.0f;
        float c = a + b;
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void reduce_0(T *input, T *output, int size)
{
    extern __shared__ T sdata[];

    int tid = threadIdx.x;
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index < size)
    {
        sdata[tid] = input[index];
    }
    else
    {
        sdata[tid] = 0;
    }
    __syncthreads();

    for (unsigned int s = 1; s < blockDim.x; s *= 2)
    {
        if (tid % (2 * s) == 0)
        {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    if (tid == 0)
    {
        output[blockIdx.x] = sdata[0];
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void reduce_1(T *input, T *output, int size)
{
    extern __shared__ T sdata[];

    int tid = threadIdx.x;
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index < size)
    {
        sdata[tid] = input[index];
    }
    else
    {
        sdata[tid] = 0;
    }
    __syncthreads();

    for (unsigned int s = 1; s < blockDim.x; s *= 2)
    {
        int index = 2 * s * tid;
        if (index < blockDim.x)
        {
            sdata[index] += sdata[index + s];
        }
        __syncthreads();
    }

    if (tid == 0)
    {
        output[blockIdx.x] = sdata[0];
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void reduce_2(T *input, T *output, int size)
{
    extern __shared__ T sdata[];

    int tid = threadIdx.x;
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index < size)
    {
        sdata[tid] = input[index];
    }
    else
    {
        sdata[tid] = 0;
    }
    __syncthreads();

    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1)
    {
        if (tid < s)
        {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    if (tid == 0)
    {
        output[blockIdx.x] = sdata[0];
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void reduce_3(T *input, T *output, int size)
{
    extern __shared__ T sdata[];

    int tid = threadIdx.x;
    int index = blockIdx.x * (blockDim.x * 2) + threadIdx.x;

    if (index + blockDim.x < size)
    {
        sdata[tid] = input[index] + input[index + blockDim.x];
    }
    else if (index + blockDim.x >= size && index < size)
    {
        sdata[tid] = input[index];
    }
    else
    {
        sdata[tid] = 0;
    }
    __syncthreads();

    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1)
    {
        if (tid < s)
        {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    if (tid == 0)
    {
        output[blockIdx.x] = sdata[0];
    }
}

template <int Block_Size_T, typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__device__ void warpReduce(volatile T *sdata, int tid)
{
    if constexpr (Block_Size_T >= 64) sdata[tid] += sdata[tid + 32];
    if constexpr (Block_Size_T >= 32) sdata[tid] += sdata[tid + 16];
    if constexpr (Block_Size_T >= 16) sdata[tid] += sdata[tid + 8];
    if constexpr (Block_Size_T >= 8) sdata[tid] += sdata[tid + 4];
    if constexpr (Block_Size_T >= 4) sdata[tid] += sdata[tid + 2];
    if constexpr (Block_Size_T >= 2) sdata[tid] += sdata[tid + 1];
}

template <int Block_Size_T, typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void reduce_4(T *input, T *output, int size)
{
    extern __shared__ T sdata[];

    int tid = threadIdx.x;
    int index = blockIdx.x * (blockDim.x * 2) + threadIdx.x;

    if (index + blockDim.x < size)
    {
        sdata[tid] = input[index] + input[index + blockDim.x];
    }
    else if (index + blockDim.x >= size && index < size)
    {
        sdata[tid] = input[index];
    }
    else
    {
        sdata[tid] = 0;
    }
    __syncthreads();

    for (unsigned int s = blockDim.x / 2; s > 32; s >>= 1)
    {
        if (tid < s)
        {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    if (tid < 32)
    {
        warpReduce<Block_Size_T>(sdata, tid);
    }

    if (tid == 0)
    {
        output[blockIdx.x] = sdata[0];
    }
}

template <int Block_Size_T, typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void reduce_5(T *input, T *output, int size)
{
    extern __shared__ T sdata[];

    int tid = threadIdx.x;
    int index = blockIdx.x * (blockDim.x * 2) + threadIdx.x;

    if (index + blockDim.x < size)
    {
        sdata[tid] = input[index] + input[index + blockDim.x];
    }
    else if (index + blockDim.x >= size && index < size)
    {
        sdata[tid] = input[index];
    }
    else
    {
        sdata[tid] = 0;
    }
    __syncthreads();

    if constexpr (Block_Size_T / 2 >= 512)
    {
        if (tid < 256)
        {
            sdata[tid] += sdata[tid + 256];
        }
        __syncthreads();
    }

    if constexpr (Block_Size_T / 2 >= 256)
    {
        if (tid < 128)
        {
            sdata[tid] += sdata[tid + 128];
        }
        __syncthreads();
    }

    if constexpr (Block_Size_T / 2 >= 128)
    {
        if (tid < 64)
        {
            sdata[tid] += sdata[tid + 64];
        }
        __syncthreads();
    }

    if (tid < 32)
    {
        warpReduce<Block_Size_T / 2>(sdata, tid);
    }

    if (tid == 0)
    {
        output[blockIdx.x] = sdata[0];
    }
}

template <int Block_Size_T, int NUM_PER_THREAD_T, typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void reduce_6(T *input, T *output, int size)
{
    extern __shared__ T sdata[];

    int tid = threadIdx.x;
    int index = blockIdx.x * (blockDim.x * NUM_PER_THREAD_T) + threadIdx.x;
    sdata[tid] = 0;

#pragma unroll
    for (int stride = 0; stride < Block_Size_T; stride += int(Block_Size_T / NUM_PER_THREAD_T))
    {
        int idTmp = index + stride;
        if(idTmp < size) sdata[tid] += input[idTmp];
    }
    //printf("%d, data1 = %d \n", tid, sdata[tid]);

    __syncthreads();

    if constexpr (Block_Size_T / NUM_PER_THREAD_T >= 512)
    {
        if (tid < 256)
        {
            sdata[tid] += sdata[tid + 256];
        }
        __syncthreads();
    }

    if constexpr (Block_Size_T / NUM_PER_THREAD_T >= 256)
    {
        if (tid < 128)
        {
            sdata[tid] += sdata[tid + 128];
            //printf("%d, data2 = %d \n", tid, sdata[tid]);
        }
        __syncthreads();
    }

    if constexpr (Block_Size_T / NUM_PER_THREAD_T >= 128)
    {
        if (tid < 64)
        {
            sdata[tid] += sdata[tid + 64];
        }
        __syncthreads();
    }

    if (tid < 32)
    {
        warpReduce<int(Block_Size_T / NUM_PER_THREAD_T)>(sdata, tid);
    }

    if (tid == 0)
    {
        output[blockIdx.x] = sdata[0];
    }
}


template <int Block_Size_T, typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__device__ void warpReduceShuffle(T *sdata, int tid)
{
    if constexpr (Block_Size_T >= 32) sdata[tid] +=__shfl_down_sync(0xffffffff,sdata[tid],16);
    if constexpr (Block_Size_T >= 16) sdata[tid] +=__shfl_down_sync(0xffffffff,sdata[tid],8);
    if constexpr (Block_Size_T >= 8) sdata[tid] +=__shfl_down_sync(0xffffffff,sdata[tid],4);
    if constexpr (Block_Size_T >= 4) sdata[tid] +=__shfl_down_sync(0xffffffff,sdata[tid],2);
    if constexpr (Block_Size_T >= 2) sdata[tid] +=__shfl_down_sync(0xffffffff,sdata[tid],1);
}

template <int Block_Size_T, typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__device__ __forceinline__ T warpReduceShuffle2(T sum)
{
    if constexpr (Block_Size_T >= 32) sum +=__shfl_down_sync(0xffffffff,sum,16);
    if constexpr (Block_Size_T >= 16) sum +=__shfl_down_sync(0xffffffff,sum,8);
    if constexpr (Block_Size_T >= 8) sum +=__shfl_down_sync(0xffffffff,sum,4);
    if constexpr (Block_Size_T >= 4) sum +=__shfl_down_sync(0xffffffff,sum,2);
    if constexpr (Block_Size_T >= 2) sum +=__shfl_down_sync(0xffffffff,sum,1);
    return sum;
}

template <int Block_Size_T, int NUM_PER_THREAD_T, typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void reduce_7(T *input, T *output, int size)
{
    extern __shared__ T sdata[];

    int tid = threadIdx.x;
    int index = blockIdx.x * (blockDim.x * NUM_PER_THREAD_T) + threadIdx.x;
    sdata[tid] = 0;

#pragma unroll
    for (int stride = 0; stride < Block_Size_T; stride += int(Block_Size_T / NUM_PER_THREAD_T))
    {
        int idTmp = index + stride;
        if(idTmp < size) sdata[tid] += input[idTmp];
    }
    //printf("%d, data1 = %d \n", tid, sdata[tid]);

    __syncthreads();

    if constexpr (Block_Size_T / NUM_PER_THREAD_T >= 512)
    {
        if (tid < 256)
        {
            sdata[tid] += sdata[tid + 256];
        }
        __syncthreads();
    }

    if constexpr (Block_Size_T / NUM_PER_THREAD_T >= 256)
    {
        if (tid < 128)
        {
            sdata[tid] += sdata[tid + 128];
            //printf("%d, data2 = %d \n", tid, sdata[tid]);
        }
        __syncthreads();
    }

    if constexpr (Block_Size_T / NUM_PER_THREAD_T >= 128)
    {
        if (tid < 64)
        {
            sdata[tid] += sdata[tid + 64];
        }
        __syncthreads();
    }

    if constexpr (Block_Size_T / NUM_PER_THREAD_T >= 64)
    {
        if (tid < 32)
        {
            sdata[tid] += sdata[tid + 32];
        }
        __syncthreads();
    }

    // if (tid < 32)
    // {
    //     warpReduceShuffle<int(Block_Size_T / NUM_PER_THREAD_T)>(sdata, tid);
    // }
    // if (tid == 0)
    // {
    //     output[blockIdx.x] = sdata[0];
    // }

    T sum = 0;

    if (tid < 32)
    {
        sum = sdata[tid];
        sum = warpReduceShuffle2<int(Block_Size_T / NUM_PER_THREAD_T)>(sum);
    }

    if (tid == 0)
    {
        output[blockIdx.x] = sum;
    }
}

template <int Block_Size_T, int NUM_PER_THREAD_T, typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void reduce_8(T *input, T *output, int size)
{
    T sum = 0;

    int tid = threadIdx.x;
    int index = blockIdx.x * (blockDim.x * NUM_PER_THREAD_T) + threadIdx.x;

#pragma unroll
    for (int stride = 0; stride < Block_Size_T; stride += int(Block_Size_T / NUM_PER_THREAD_T))
    {
        int idTmp = index + stride;
        if(idTmp < size) sum += input[idTmp];
    }
    //printf("%d, data1 = %d \n", tid, sdata[tid]);

    __syncthreads();
    // WARP_SIZE = blockDim.x / 32; WARP_SIZE = BLOCK_SIZE / NUM_PER_THREAD / 32;
    static __shared__ T warpLevelSums[WARP_SIZE]; // 2048 / 32 = 64 not valid!
    const int laneId = threadIdx.x % 32; //warp 0 0, 1, 2, 3... warp 1 0, 1, 2, 3...
    const int warpId = threadIdx.x / 32; //warp 0, warp 1, warp 2, warp 3...

    sum = warpReduceShuffle2<int(Block_Size_T / NUM_PER_THREAD_T)>(sum);

    if (laneId == 0)
    {
        warpLevelSums[warpId] = sum;
    }
    __syncthreads();

    assert(WARP_SIZE <= 32);
    sum = (threadIdx.x < WARP_SIZE) ? warpLevelSums[laneId] : 0;

    assert(WARP_SIZE <= 32);
    if (warpId == 0)
    {
        sum = warpReduceShuffle2<int(WARP_SIZE)>(sum);
    }

    if (tid == 0)
    {
        output[blockIdx.x] = sum;
    }
}


int main()
{
    TYPE h_input[N];
    for (int i = 0; i < N; ++i)
    {
        h_input[i] = 1; // 初始化数组为1
    }

    thrust::device_vector<TYPE> d_input(h_input, h_input + N);
    thrust::device_vector<TYPE> d_output(ceil(N / (BLOCK_SIZE * 1.0)), 0);

    int threads_per_block = BLOCK_SIZE;
    int no_of_blocks = (N + threads_per_block - 1) / threads_per_block;

    reduce_0<<<no_of_blocks, threads_per_block, BLOCK_SIZE * sizeof(TYPE)>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));
    reduce_1<<<no_of_blocks, threads_per_block, BLOCK_SIZE * sizeof(TYPE)>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));
    reduce_2<<<no_of_blocks, threads_per_block, BLOCK_SIZE * sizeof(TYPE)>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));
    reduce_3<<<no_of_blocks, threads_per_block / 2, BLOCK_SIZE / 2 * sizeof(TYPE)>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));
    reduce_4<int(BLOCK_SIZE)><<<no_of_blocks, threads_per_block / 2, BLOCK_SIZE / 2 * sizeof(TYPE)>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));
    reduce_5<int(BLOCK_SIZE)><<<no_of_blocks, threads_per_block / 2, BLOCK_SIZE / 2 * sizeof(TYPE)>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));
    reduce_6<int(BLOCK_SIZE), int(NUM_PER_THREAD)><<<no_of_blocks, threads_per_block / NUM_PER_THREAD, BLOCK_SIZE / NUM_PER_THREAD * sizeof(TYPE)>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));
    reduce_7<int(BLOCK_SIZE), int(NUM_PER_THREAD)><<<no_of_blocks, threads_per_block / NUM_PER_THREAD, BLOCK_SIZE / NUM_PER_THREAD * sizeof(TYPE)>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));
    reduce_8<int(BLOCK_SIZE), int(NUM_PER_THREAD)><<<no_of_blocks, threads_per_block / NUM_PER_THREAD>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));

    thrust::host_vector<TYPE> h_output = d_output;

    // int final_sum = thrust::reduce(d_input.begin(), d_input.end(), 0, thrust::plus<int>());

    int final_sum = 0;
    for (int i = 0; i < h_output.size(); ++i)
    {
        final_sum += h_output[i];
    }

    std::cout << "Sum: " << final_sum << std::endl;

    return 0;
}

In [ ]:
!nvcc -o reduction -lineinfo reduction.cu

In [ ]:
!./reduction

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2024.2.3_2024.2.3.38-1_amd64.deb
!apt update
!apt install ./nsight-systems-2024.2.3_2024.2.3.38-1_amd64.deb
!apt --fix-broken install

In [ ]:
!nsys profile -o report_nsys_reduction ./reduction -f

In [ ]:
!ncu --set full --replay-mode kernel --target-processes all -o report_ncu_reduction -f ./reduction